## Weight & Biases
Hyperparameter Tuning mit wandb

### Sweep Config
Konfiguration des wand sweep mit den gewünschten Hyperparameter

In [1]:
import wandb

wandb.login()

sweep_config = {
  'name' : 'GradientBoosting-soft-rain-64',
  'method' : 'random',
  'parameters' : {
    'n_estimators' : {
      'values' : [16, 32, 64, 128]
    },
    'learning_rate' :{
      'min': 0.05,
      'max': 0.2
    },
    'subsample' :{
      'min': 0.5,
      'max': 1.0
    },
    'max_depth' :{
      'values' : [2, 3, 4, 8]
    },
    'min_samples_split' :{
      'values' : [2, 4]
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project='Metriken Bauwesen Sweeps', entity='devcore')

wandb: Currently logged in as: devcore (use `wandb login --relogin` to force relogin)
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. {'min': 0.5, 'max': 1} is not valid under any of the given schemas


Create sweep with ID: mkxhrc96
Sweep URL: https://wandb.ai/devcore/Metriken%20Bauwesen%20Sweeps/sweeps/mkxhrc96


### Run Sweep
Sweep auf Agent laufen lassen für Hyperparameter-Tuning und Logging auf wandb

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate, RepeatedKFold

import src.package.importer as im
import src.package.ml_helper as ml_helper
from numpy import mean

def train():
    config = {'model': 'GradientBoost', 'run': 'soft-rain-64'}
    with wandb.init(config=config):
        # load dataset
        df = im.get_extended_dataset('../package/datasets/train_set.csv')
        df = ml_helper.remove_outliers(df, factor=2.3)
        X, y = ml_helper.hnf_dataset_full(df)

        # model with hyperparameter
        config = wandb.config
        regr = GradientBoostingRegressor(random_state=0,
                                         learning_rate=config.learning_rate,
                                         subsample=config.subsample,
                                         n_estimators=config.n_estimators,
                                         max_depth=config.max_depth,
                                         min_samples_split=config.min_samples_split,)

        # evaluate with cv and log to wandb
        cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
        scores = cross_validate(regr, X, y, cv=cv, scoring=['r2'])
        print(scores)
        wandb.log({'r2': mean(scores['test_r2'])})

wandb.agent(sweep_id, train, count=60)

wandb: Agent Starting Run: xupga44j with config:
wandb: 	learning_rate: 0.19370257214729453
wandb: 	max_depth: 3
wandb: 	min_samples_split: 4
wandb: 	n_estimators: 128
wandb: 	subsample: 0.8063601064757892
wandb: Agent Starting Run: a3stldt9 with config:
wandb: 	learning_rate: 0.18569948777175427
wandb: 	max_depth: 4
wandb: 	min_samples_split: 4
wandb: 	n_estimators: 16
wandb: 	subsample: 0.6238172005750426
wandb: Agent Starting Run: 1islracz with config:
wandb: 	learning_rate: 0.12156237106177313
wandb: 	max_depth: 4
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 128
wandb: 	subsample: 0.6441243479406173
wandb: Agent Starting Run: fq94rr9s with config:
wandb: 	learning_rate: 0.17538038251053245
wandb: 	max_depth: 2
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 16
wandb: 	subsample: 0.7324948562671731
wandb: Agent Starting Run: t1ych05d with config:
wandb: 	learning_rate: 0.11541919700331116
wandb: 	max_depth: 4
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 64
wandb: 	sub

{'fit_time': array([0.04800415, 0.05702233, 0.05103016, 0.047997  , 0.047997  ,
       0.047997  , 0.04699254, 0.04599214, 0.0460012 , 0.04700518,
       0.04600072, 0.0479722 , 0.04599881, 0.04702854, 0.04697466]), 'score_time': array([0.00197339, 0.00096965, 0.00100374, 0.00200224, 0.00200272,
       0.00200057, 0.00099969, 0.00200009, 0.00200129, 0.00100088,
       0.00199866, 0.00199986, 0.00199986, 0.00099206, 0.00199986]), 'test_r2': array([0.95087467, 0.95106688, 0.96372886, 0.93077058, 0.88066625,
       0.95596745, 0.95539413, 0.98381326, 0.93973221, 0.90603569,
       0.94517859, 0.97002975, 0.95901264, 0.97221447, 0.92868023])}
{'fit_time': array([0.00699997, 0.00799942, 0.0079999 , 0.00699973, 0.00800014,
       0.0079999 , 0.00900149, 0.00799322, 0.00899816, 0.00800109,
       0.00797057, 0.00803065, 0.00699973, 0.00799036, 0.01101589]), 'score_time': array([0.00200009, 0.0010004 , 0.00100017, 0.00200009, 0.00100017,
       0.00099993, 0.00100136, 0.00200057, 0.00099897, 0

r2,0.94621
_runtime,2
_timestamp,1624376611
_step,0


r2,▁
_runtime,▁
_timestamp,▁
_step,▁


r2,0.94347
_runtime,1
_timestamp,1624376616
_step,0


...

r2,▁
_runtime,▁
_timestamp,▁
_step,▁


r2,0.94442
_runtime,1
_timestamp,1624376968
_step,0


r2,▁
_runtime,▁
_timestamp,▁
_step,▁


r2,0.91566
_runtime,2
_timestamp,1624376974
_step,0


r2,▁
_runtime,▁
_timestamp,▁
_step,▁
